Подсчет темпа генерации для пульсаров

In [2]:
import os
import sys
import glob
import datetime
from functools import reduce

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [4]:
pulsar_name = '1133+16'
gp_table_pulsar = pd.read_table(pulsar_name + '_gp_kaz_10.csv', sep='\t')
session_pulsar = pd.read_table(pulsar_name + '_obs_kaz.csv', sep='\t')

/home/andr/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


FileNotFoundError: [Errno 2] File b'1133+16_gp_kaz_10.csv' does not exist: b'1133+16_gp_kaz_10.csv'

In [0]:
gp_table_pulsar['Time_series'] = pd.to_datetime(gp_table_pulsar['Date'], format='%d.%m.%Y', errors='ignore')
session_pulsar['Time_series'] = pd.to_datetime(session_pulsar['Date'], format='%d.%m.%Y', errors='ignore')
gp_table_pulsar = gp_table_pulsar.set_index('Time_series')
session_pulsar = session_pulsar.set_index('Time_series')

In [0]:
gp_table_pulsar.head()

,Date,Time start,"Tay, ms","Period, s","Numpointwin, point","Numpulse, a.u.","Median, adc u","STD, adc u",File name,"Count of GP, u",Num pulse,"point of gp, point","amp of gp, adc u","W50, point","W10, point",path plot
Time_series,,,,,,,,,,,,,,,,
2016-09-01,01.09.2016,13:19:00.8392366,1.2288,1.187933,570,167,47.088339,0.966374,010916_1133+16_00_profiles.txt,1,98,147,37.491761,1.99150,4.58045,./gp_plot_pulsar/01.09.2016_plot_1133+16_98.png
2017-09-01,01.09.2017,13:20:01.5297752,1.2288,1.187933,570,167,45.488041,0.936635,010917_1133+16_00_profiles.txt,1,39,123,615.645059,1.93460,4.69425,./gp_plot_pulsar/01.09.2017_plot_1133+16_39.png
2017-09-01,01.09.2017,13:20:01.5297752,1.2288,1.187933,570,167,45.488041,0.867013,010917_1133+16_00_profiles.txt,1,71,148,855.178459,1.90615,8.96175,./gp_plot_pulsar/01.09.2017_plot_1133+16_71.png
2016-10-01,01.10.2016,11:21:03.5212043,1.2288,1.187875,570,167,37.924473,0.769368,011016_1133+16_00_profiles.txt,1,49,138,74.521227,2.21910,5.46240,./gp_plot_pulsar/01.10.2016_plot_1133+16_49.png
2016-12-01,01.12.2016,07:21:13.8233013,1.2288,1.187801,570,167,41.978878,0.859004,011216_1133+16_00_profiles.txt,1,83,154,98.017922,1.90615,3.18640,./gp_plot_pulsar/01.12.2016_plot_1133+16_83.png


In [0]:
session_pulsar.head()

,Date,Session
Time_series,,
2016-01-01,01.01.2016,1
2017-01-01,01.01.2017,1
2017-02-01,01.02.2017,1
2016-03-01,01.03.2016,1
2017-03-01,01.03.2017,1


In [0]:
gp_table_pulsar['Count of GP, u'].resample('Y').sum()

Time_series
2013-12-31     5
2014-12-31    33
2015-12-31    24
2016-12-31    57
2017-12-31    63
2018-12-31     5
Freq: A-DEC, Name: Count of GP, u, dtype: int64

In [0]:
session_pulsar['Session'].resample('Y').sum()

Time_series
2013-12-31     30
2014-12-31     93
2015-12-31    159
2016-12-31    183
2017-12-31    264
2018-12-31     58
Freq: A-DEC, Name: Session, dtype: int64

In [0]:
gp_pulsar_m = gp_table_pulsar['Count of GP, u'].resample('M').sum().to_frame()

In [0]:
gp_pulsar_m_ses = session_pulsar['Session'].resample('M').sum().to_frame()

In [0]:
result_pulsar = gp_pulsar_m.join(gp_pulsar_m_ses, how='outer')

In [0]:
result_pulsar.head()

,"Count of GP, u",Session
Time_series,,
2013-10-31,1,7
2013-11-30,1,9
2013-12-31,3,14
2014-01-31,2,10
2014-02-28,2,11


In [0]:
result_pulsar['pace_GP']  = np.divide(result_pulsar['Count of GP, u'], result_pulsar['Session'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [0]:
result_pulsar.head()

,"Count of GP, u",Session,pace_GP
Time_series,,,
2013-10-31,1,7,0.142857
2013-11-30,1,9,0.111111
2013-12-31,3,14,0.214286
2014-01-31,2,10,0.200000
2014-02-28,2,11,0.181818


In [0]:
#plt.close()
#ax = result_pulsar.reset_index().plot.bar(x='Time_series', y='pace_GP', figsize=(25, 8), color='b')
## ax
#ax.set_title(pulsar_name, fontsize = 20.0)
#ax.set_ylabel('Pace of giant pulses, counts/month', fontsize = 20.0)
#path_ax = ('C:\\astrophysics\\' + pulsar_name + '_pace_of_gp' + '.png')
#plt.savefig(path_ax, format = 'png', dpi = 300, bbox_inches='tight')

In [0]:
#plt.close()
#ax = result_pulsar.reset_index().plot.bar(x='Time_series', y=['Session', 'pace_GP'], figsize=(25, 8), stacked=False, width=0.95)
## ax
#ax.set_title(pulsar_name, fontsize = 20.0)
#ax.set_ylabel('Pace of generation of GP/Pace of sessions, counts/month', fontsize = 20.0)
#path_ax = ('C:\\astrophysics\\' + pulsar_name + '_sessions&pace' + '.png')
#plt.savefig(path_ax, format = 'png', dpi = 300, bbox_inches='tight')

In [0]:
#a = [result_pulsar['pace_GP'], result_pulsar['Session']]

In [0]:
#a.plot.bar()

In [0]:
#len(result_pulsar['pace_GP'])

In [0]:
max(result_pulsar['pace_GP'])


0.9375

In [0]:
min(result_pulsar['pace_GP'])

0.0

In [0]:
sorted(result_pulsar['pace_GP'])

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.043478260869565216,
 0.06666666666666667,
 0.06666666666666667,
 0.08333333333333333,
 0.09090909090909091,
 0.0967741935483871,
 0.1,
 0.1111111111111111,
 0.1111111111111111,
 0.125,
 0.125,
 0.13043478260869565,
 0.14285714285714285,
 0.14285714285714285,
 0.14814814814814814,
 0.16666666666666666,
 0.18181818181818182,
 0.2,
 0.2,
 0.20689655172413793,
 0.21052631578947367,
 0.21428571428571427,
 0.2222222222222222,
 0.24,
 0.2413793103448276,
 0.25,
 0.25925925925925924,
 0.2692307692307692,
 0.27586206896551724,
 0.3333333333333333,
 0.3333333333333333,
 0.36363636363636365,
 0.36363636363636365,
 nan,
 0.2222222222222222,
 0.375,
 0.6296296296296297,
 0.9375,
 nan,
 nan,
 0.35714285714285715,
 nan,
 nan,
 nan,
 0.375,
 0.5,
 0.7,
 0.9]

### Темпы генерации
- **0809+74** - 0,00 - 0,57
- **0950+08** - 0,25 - 11,83
- **1112+50** - 0,00 - 9,625
- **1133+16** - 0,00 - 0,938
- **1237+25** - 0,00 - 1,700